In [105]:
import torch
import pandas as pd
import re
import sklearn
from sklearn.neighbors import NearestNeighbors

from utils.splitters import Chunker

import nltk
nltk.download('punkt')
torch.cuda.is_available()

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [107]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3", device='cuda')
model.max_seq_length = 512

In [254]:
texts = pd.read_csv('documents (1).csv', header=None, names=['url','text'])
texts = texts.dropna()
texts['text'] = texts['text'].apply(lambda x: re.sub(' +', ' ', x))
texts['text'] = texts['text'].apply(lambda x: re.sub('\n+', '\n', x))
texts.url = texts.url.apply(lambda x: x if  x[-1] == '/' else x+'/')
texts.text.apply(lambda x: x.replace(x[x.find('<!--'): x.rfind('-->')+3], '').replace('Complex', '').strip('\n '))
texts = texts.set_index('url')

In [255]:
queries = pd.read_csv('Book1 (1).csv', sep=';')
queries.site_id = queries.site_id.apply(lambda x: x if  x[-1] == '/' else x+'/')
queries.site_id = queries.site_id.apply(lambda x: x if 'http://pravo.gov.ru' in x else 'https://cbr.ru' + x)

In [256]:
chunker = Chunker(max_chunk_len=2500, overlap_len=500)

In [257]:
chunks, urls = chunker.split_texts(texts)

In [258]:
len(chunks)

35230

In [259]:
embeddings = model.encode(chunks, batch_size=32, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/1101 [00:00<?, ?it/s]

In [260]:
import joblib

In [261]:
joblib.dump(embeddings, 'bge-m3-new-clear.pkl')

['bge-m3-new-clear.pkl']

In [262]:
embeddings.shape

(35230, 1024)

In [263]:
knn = NearestNeighbors(metric='cosine')
knn.fit(embeddings)

NearestNeighbors(metric='cosine')

In [264]:
import numpy as np

In [276]:
queries.question.tolist()

['Кто может принимать участие в закупках Банка России?',
 'Cроки предоставления информации об активах и обязательствах резидентом-экспортером',
 'Что считается уровнем риска?',
 'Какими видами рисков обязательно должен управлять кредитный кооператив?',
 'На какие банковские операции распространяется сниженная комиссия для участников СВО?',
 'Регионы проведения эксперимента по партнерскому финансированию',
 'Что такое ВЕБ.РФ?',
 'Какие полномочия у председателя ВЭБ.РФ?',
 'Какие условия совершения операций по выдаче микрозаймов микрофинансовой организацией?']

In [277]:
queries.site_id

0           https://cbr.ru/Crosscut/LawActs/File/6620/
1    https://cbr.ru/Queries/UniDbQuery/File/90134/2...
2     https://cbr.ru/Queries/UniDbQuery/File/90002/12/
3     https://cbr.ru/Queries/UniDbQuery/File/90002/12/
4           https://cbr.ru/Crosscut/LawActs/File/6144/
5    http://pravo.gov.ru/proxy/ips/?docbody=&prevDo...
6    http://pravo.gov.ru/proxy/ips/?docbody=&nd=102...
7    http://pravo.gov.ru/proxy/ips/?docbody=&nd=102...
8     https://cbr.ru/Queries/UniDbQuery/File/90002/49/
Name: site_id, dtype: object

In [275]:
print('Site found in Neighbours')
for i in range(1,20,2):
    skores, neighbors = knn.kneighbors(model.encode(queries.question, batch_size=32, normalize_embeddings=True), i)
    num_true = 0
    for ind, line in enumerate(neighbors):
        if queries.site_id[ind] in np.array(urls)[line]:
            num_true += 1
        else:
            print(queries.question[ind])
        
    print(f'hits@{i}:', num_true/len(queries))

Site found in Neighbours
Что считается уровнем риска?
hits@1: 0.8888888888888888
Что считается уровнем риска?
hits@3: 0.8888888888888888
Что считается уровнем риска?
hits@5: 0.8888888888888888
Что считается уровнем риска?
hits@7: 0.8888888888888888
Что считается уровнем риска?
hits@9: 0.8888888888888888
Что считается уровнем риска?
hits@11: 0.8888888888888888
hits@13: 1.0
hits@15: 1.0
hits@17: 1.0
hits@19: 1.0
